- With Stucture Output Supported LLMs
- Without (Output Parsers)

In [1]:
from dotenv import load_dotenv
load_dotenv() 

from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash-preview-04-17")

## with Pydantic

In [2]:
from pydantic import BaseModel, Field

class Weather(BaseModel):
    temperature: int = Field(...,description="Temperature of the day")

In [3]:
structure = llm.with_structured_output(Weather)

str_output = structure.invoke("its so harsh to go outside, as its hot as 50 degree")

In [7]:
str_output.model_dump()

{'temperature': 50}

In [13]:
# optional output
from typing import Literal, Optional

class Weather(BaseModel):
    temperature: int = Field(..., gt=-100, lt=100 ,description="Temperature of the day")
    name: Optional[str] = Field(None,  description="Name of the person")
    
structure = llm.with_structured_output(Weather)
str_output = structure.invoke("kamal! its so harsh to go outside, as its hot as 500 degree")
str_output

ValidationError: 1 validation error for Weather
temperature
  Input should be less than 100 [type=less_than, input_value=500.0, input_type=float]
    For further information visit https://errors.pydantic.dev/2.11/v/less_than

## with Json

In [ ]:
# format

{
    "title": "student",
    "description": "schema about students",
    "types": "object",
    "properties": {
        "name": "string",
        "age": "integer"
    },
    "required": ["name"]
}

In [2]:
json_schema = {
  "title": "Review",
  "type": "object",
  "properties": {
    "key_themes": {
      "type": "array",
      "items": {
        "type": "string"
      },
      "description": "Write down all the key themes discussed in the review in a list"
    },
    "summary": {
      "type": "string",
      "description": "A brief summary of the review"
    },
    "sentiment": {
      "type": "string",
      "enum": ["pos", "neg"],
      "description": "Return sentiment of the review either negative, positive or neutral"
    },
    "pros": {
      "type": ["array", "null"],
      "items": {
        "type": "string"
      },
      "description": "Write down all the pros inside a list"
    },
    "cons": {
      "type": ["array", "null"],
      "items": {
        "type": "string"
      },
      "description": "Write down all the cons inside a list"
    },
    "name": {
      "type": ["string", "null"],
      "description": "Write the name of the reviewer"
    }
  },
  "required": ["key_themes", "summary", "sentiment"]
}


structured_model = llm.with_structured_output(json_schema)

result = structured_model.invoke("""I recently upgraded to the Samsung Galaxy S24 Ultra, and I must say, it’s an absolute powerhouse! The Snapdragon 8 Gen 3 processor makes everything lightning fast—whether I’m gaming, multitasking, or editing photos. The 5000mAh battery easily lasts a full day even with heavy use, and the 45W fast charging is a lifesaver.

The S-Pen integration is a great touch for note-taking and quick sketches, though I don't use it often. What really blew me away is the 200MP camera—the night mode is stunning, capturing crisp, vibrant images even in low light. Zooming up to 100x actually works well for distant objects, but anything beyond 30x loses quality.

However, the weight and size make it a bit uncomfortable for one-handed use. Also, Samsung’s One UI still comes with bloatware—why do I need five different Samsung apps for things Google already provides? The $1,300 price tag is also a hard pill to swallow.

Pros:
Insanely powerful processor (great for gaming and productivity)
Stunning 200MP camera with incredible zoom capabilities
Long battery life with fast charging
S-Pen support is unique and useful
                                 
Review by Nitish Singh
""")

result

Key 'parameters' is not supported in schema, ignoring


[{'args': {'sentiment': 'pos',
   'cons': 'weight and size make one-handed use uncomfortable',
   'summary': 'The Samsung Galaxy S24 Ultra is a powerful phone with an excellent processor, long-lasting battery, and a stunning 200MP camera. However, its size makes one-handed use difficult, it has bloatware, and the price is high. Zoom quality also degrades past 30x.',
   'key_themes': ['processor',
    'battery',
    'charging',
    'S-Pen',
    'camera',
    'size and weight',
    'software',
    'price']},
  'type': 'Review'}]

## 🚀 When to Use What?

| Feature                  | TypedDict ✅ | Pydantic 🚀 | JSON Schema 🌐 |
|--------------------------|--------------|-------------|----------------|
| Basic structure          | ✅           | ✅          | ✅             |
| Type enforcement         | ✅           | ✅          | ✅             |
| Data validation          | ❌           | ✅          | ✅             |
| Default values           | ❌           | ✅          | ❌             |
| Automatic conversion     | ❌           | ✅          | ❌             |
| Cross-language compatibility | ❌       | ❌          | ✅             |
